# INFO371 Problem set 4: k-NN, TF-IDF

## Introduction

In this project, we will program k-NN with cosine similarity using both bag-of-words and TF-IDF approaches.

We will use two types of data: first, various texts from Cantenbury corpus with several books added from
Project Gutenberg, and thereafter Rotten Tomatoes, brief movie reviews. In case of the texts, we will find the correct source of the text. In case of tomatoes, to predict if the review is for a "rotten" or"fresh" movie.

## 1 Where are these texts coming from?

The data file texts.csv contains the textswe need to classify. It contains the following variables:

- **name**: name of the original file. It is usually author-name form and it should be fairly easy to find the
original text in most cases.
- **size**: size of the original text, in bytes
- **lines**: size of the original text, in lines
- **chunkid**: chunk id, from 1 and growing, see chunk. If you want to re-assemble the original texts, you just have to put these next to each other in the order of chunkid.
- **chunk**: a page of text. It is not really a page, just 25 lines of text, whatever happened to be on those 25
lines.

Text chunks are just verbatim texts that may contain all kind of characters, including newlines. Note the
file is tab separated and uses quotes for strings.

We need to read all the texts, convert these to a) bag-of-words, and b) TF-IDF-s, and predict the
correct source using k-NN.

### 1.1 Bag of words

First, let's use bag-of-words (BOW) approach.

1. Load the data. Print out a few lines of it to inspect it's structure.
2. Inspect some of the texts. Note that chunkid 1 corresponds to the first page of the text.
3. List all the text sources listed in variable name. How many different texts does the data contain?

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
texts = pd.read_csv("texts.csv", sep='\t')
texts.head()

,name,size,lines,chunkid,chunk
0,balbulus-early-life-charlemagne,259062,4394,1,\r\nTitle: Early Lives of Charlemagne by Eginh...
1,balbulus-early-life-charlemagne,259062,4394,2,"\r\n\r\nThe notes, keyed to line numbers in th..."
2,balbulus-early-life-charlemagne,259062,4394,3,\r\n From a bronze statuette in the Mu...
3,balbulus-early-life-charlemagne,259062,4394,4,\r\n _A lui finit la dissolutio...
4,balbulus-early-life-charlemagne,259062,4394,5,public opinion in regard to the meaning of fal...


In [3]:
print("Number of texts: %i" % texts.shape[0])
print("Number of titles: %i" % len(np.unique(texts.name.astype('str'))))

Number of texts: 12924
Number of titles: 29


In [4]:
first_pages = texts[texts['chunkid'] == 1]
first_pages

# inspect alice in wonderland
print(first_pages[first_pages['name'] == 'carroll-alice-wonderland'].chunk.values[0])

                ALICE'S ADVENTURES IN WONDERLAND

                          Lewis Carroll

               THE MILLENNIUM FULCRUM EDITION 2.9




                            CHAPTER I

                      Down the Rabbit-Hole


  Alice was beginning to get very tired of sitting by her sister
on the bank, and of having nothing to do:  once or twice she had
peeped into the book her sister was reading, but it had no
pictures or conversations in it, `and what is the use of a book,'
thought Alice `without pictures or conversation?'

  So she was considering in her own mind (as well as she could,
for the hot day made her feel very sleepy and stupid), whether
the pleasure of making a daisy-chain would be worth the trouble


In [5]:
print("All text sources: ", np.unique(texts.name), "\n")
print("Number of text sources: ", len(np.unique(texts.name)))

All text sources:  ['balbulus-early-life-charlemagne' 'beesly-queen-elizabeth' 'bible'
 'carroll-alice-wonderland' 'chipman-earliest-electromagnetic-instruments'
 'cia-world-factbook-1992' 'eckstein-quintus-claudius'
 'fisher-quaker-colonies' 'gallienne-quest-of-golden-girl'
 'gordon-quiet-talks-crowned-christ' 'hardy-madding-crowd'
 'infiltrating-open-systems' 'kant-metaphysical-elements-ethics'
 'karn-snowflakes' 'milton-paradise-lost'
 'naval-academy-sound-military-decision' 'newsgroup'
 'paper-compact-hash-tables' 'paper-data-compression'
 'paper-logical-implementation-of-arithmetic'
 'paper-programming-by-example' 'paper-search-for-autonomy'
 'selected-polish-tales' 'shakespeare-as-you-like-it'
 'unamuno-tragic-sense-of-life' 'vaneeden-quest'
 'webster-early-european-history' 'why-speech-output'
 'workshop-proceedings'] 

Number of text sources:  29


Now it's time to split it into testing and validating parts.

**Warning**: The dataset contains 13,000 texts and 65,000 unique words. If you load in
everything, the BOW matrix takes approximately 8GB of RAM. It is recommended to start with very small
datasets, say 10 pages for both training and validation.

In [6]:
from sklearn.model_selection import train_test_split


ntrain = 40
nVal = 10

train = texts.sample(n=ntrain, random_state=1)
val = texts.loc[~texts.index.isin(train)].sample(n=nVal, random_state=1)

# split intro train/val
X_train = train.chunk
X_val = val.chunk
y_train = train.name
y_val = val.name

After creating the training and testing sets, it is time to create the dictionary. This process contains two steps: 

1) Create a dictionary of all your texts. 

2) Recode all the texts as BOW vector.


In [7]:
sentences = np.concatenate((X_train.values, X_val.values),axis=0)

# initialize the vectorizer
vectorizer = CountVectorizer(min_df=0)

# create the dictionary
vectorizer.fit(sentences) # `fit` builds the vocabulary

# transform your data into the BOW array
X = vectorizer.transform(sentences).toarray()

print(X, '\n')
print('Number of rows:', len(X))

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]] 

Number of rows: 50


Now we have all your (training and validation) texts in BOW form. This is great; we just transformed texts into (numeric) vectors! Now we need to implement cosine similarity between these vectors.

We should write a function that takes in two vectors, x and y, and returns the corresponding cosine similarity c(x,y). Test if c(x, x) = 1, test, and also a few other vectors.

In [8]:
def cos_sim(a, b):
    """Takes 2 vectors a, b and returns the cosine similarity according to the definition of the dot product"""
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b)

cos_sim(X[1], X[1])

1.0000000000000002

Finally, we will implement k-NN (without using pre-existing libraries!). Let's first try 1-NN, and when this works, extend it to k-NN. The algorithm is something along these lines:
    
(a) Pick a validation case y (later we will loop over all of these).

(b) For each vector in the training set xi, compute the cosine similarity c(y, xi). Store this number,
and ensure we know which xi corresponds to each c value.

(c) Now order all the cosine similarities we just computed in an increasing order.

(d) Pick the k highest c-s. These correspond to our k nearest neighbors! Ensure we know which
texts these are.

Organize a majority voting among them to learn which text is the most popular among them. Create a frequency table and pick the most common text source name based on this. Then, compute accuracy (percentage of correct predictions). How good is your algorithm?

In [9]:
y = X[-1]
train_cases = X[:40]
result = []
correct = []

for i,j in enumerate(train_cases):
    result.append((i, cos_sim(y, j)))
    
ordered_result = sorted(result, key=lambda tup: tup[1], reverse=True)  

print("Predicted:", train.iloc[ordered_result[0][0], 0])
print("Correct:", val.iloc[9, 0])

Predicted: webster-early-european-history
Correct: webster-early-european-history


In [10]:
# get accuracy of predicted responses
def accuracy(responses):
    return float(sum(i == 1 for i in responses)) / float(len(responses))

In [11]:
ntrain = 1000
nVal = 100

train = texts.sample(n=ntrain,random_state = 123)
val = texts.loc[~texts.index.isin(train)].sample(n=nVal,random_state = 123)

X_train = train.chunk
X_val = val.chunk
y_train = train.name
y_val = val.name

sentences = np.concatenate((X_train.values, X_val.values),axis=0)

vectorizer = CountVectorizer(min_df=0)
vectorizer.fit(sentences)
X = vectorizer.transform(sentences).toarray()

In [12]:
train_cases = X[:ntrain]
correct = []

for i in range(ntrain, ntrain + nVal):
    val_case = X[i]
    result = []
    
    for j,k in enumerate(train_cases):
        result.append((j, cos_sim(k, val_case)))
        
    ordered_result = sorted(result, key=lambda tup: tup[1], reverse=True)    
    # add 1 if prediction is correct
    if train.iloc[ordered_result[0][0], 0] == val.iloc[i-1000, 0]:
        correct.append(1)
    else:
        correct.append(0)
    

print("Accuracy:", accuracy(correct))

Accuracy: 1.0


Now that we've tried this, let's look into other values for k!

In [13]:
def knn(k):
    print("k:", k)
    correct = []
    
    for i in range(ntrain, ntrain + nVal):
        val_case = X[i]
        result = []
        
        for j, m in enumerate(train_cases):
            result.append((j, cos_sim(m, val_case)))
            
        ordered_result = sorted(result, key=lambda tup: tup[1],reverse=True)[:k]
        vals = [y_train.iloc[o[0]] for o in ordered_result]
        most_common = max(map(lambda val: (vals.count(val), val), set(vals)))[1]
        
        if str(most_common) == y_val.iloc[i-ntrain]:
            correct.append(1)
        else:
            correct.append(0)
    print("Accuracy:", accuracy(correct))

In [14]:
import time
for l in [1,5,25]:
    start = time.time()
    knn(l)
    end = time.time()
    print("Time elapsed:",end-start)

k: 1
Accuracy: 1.0
Time elapsed: 6.447309255599976
k: 5
Accuracy: 0.64
Time elapsed: 6.572238445281982
k: 25
Accuracy: 0.44
Time elapsed: 6.905047178268433


Our best model is when **k = 1** with an accuracy of 1.0. Accuracy decreases as we increase k.

### 1.2 TF-IDF

BOW is great but could be even greater. Can we get a better result using TF-IDF? TF-IDF is simply a way how to weigh the word frequency in a more informative way.

1. Implement TF-IDF transformation (without rely on existing libraries!). This involves manipulating the training and validation data matrices, nothing else needs to be done.
2. Ensure that the cosine similarity implemented earlier also works for vectors in TF-IDF form. It should without any modifcations.
3. Run k-NN with the cosine similarity algorithm again, using several k values. The algorithm should not need any modifcations.
4. How accurate is BOW versus TF-IDF? How does choice of k change the results? Is BOW or TF-IDF faster to run?

In [15]:
def knn_tfidf(k):
    print("k:", k)
    correct = []
    for i in range(ntrain, ntrain + nVal):
        val_case = tfidf[i]
        result = []
        
        for j, m in enumerate(train_tfidf):
            result.append((j, cos_sim(m, val_case)))
            
        ordered_result = sorted(result, key=lambda tup: tup[1], reverse=True)[:k]
        vals = [y_train.iloc[o[0]] for o in ordered_result]
        most_common = max(map(lambda val: (vals.count(val), val), set(vals)))[1]
        
        if str(most_common) == y_val.iloc[i-ntrain]:
            correct.append(1)
        else:
            correct.append(0)
            
    print("Accuracy:", accuracy(correct))

In [16]:
tf = X.T #term frequency

idf = np.log(tf.shape[1] / ( 1 + sum(tf != 0) ))
idf = np.diag(idf)

tfidf = np.dot(tf,idf).T

print(tf.shape)
print(idf.shape)
print(tfidf.shape, '\n')

# normalize
tfidf = tfidf/np.sqrt(np.sum(tfidf**22))

# train datasets
train_tfidf = tfidf[:ntrain]

for l in [1, 5, 25]:
    start = time.time()
    knn_tfidf(l)
    end = time.time()
    print("Time elapsed:", end-start)

(19029, 1100)
(1100, 1100)
(1100, 19029) 

k: 1
Accuracy: 1.0
Time elapsed: 40.82515239715576
k: 5
Accuracy: 0.64
Time elapsed: 40.05109643936157
k: 25
Accuracy: 0.44
Time elapsed: 40.560301065444946


*How accurate is BOW versus TF-IDF? How does choice of k change the results? Is BOW or TF-IDF faster to run?*

Accuracy appears to be the same for both models when k = 1, 5, and 25. The change in the choice of k remains to have the same pattern, where accuracy decreases as k increases. BOW is much faster to run than TD-IDF. 

## 2. Are tomatoes fresh or rotten?

Our next task is to classify the Rotten Tomatoes movie reviews. Briefly, approved critics can write reviews for movies, and evaluate the movie as "fresh" or "rotten". The webpage normally shows a short quote from each critic, and whether it was evaluated as fresh or rotten.

 Load the data. Inspect it a little bit to see how it is coded and organized. How many cases are there?

In [17]:
rotten = pd.read_csv("reviews.csv")
print("Number of cases:", len(rotten))
rotten.head()

Number of cases: 13442


,critic,fresh,imdb,link,publication,quote,review_date,rtid,title
0,Derek Adams,fresh,114709,http://www.timeout.com/film/reviews/87745/toy-...,Time Out,"So ingenious in concept, design and execution ...",2009-10-04 00:00:00,9559,Toy Story
1,Richard Corliss,fresh,114709,"http://www.time.com/time/magazine/article/0,91...",TIME Magazine,The year's most inventive comedy.,2008-08-31 00:00:00,9559,Toy Story
2,David Ansen,fresh,114709,http://www.newsweek.com/id/104199,Newsweek,A winning animated feature that has something ...,2008-08-18 00:00:00,9559,Toy Story
3,Leonard Klady,fresh,114709,http://www.variety.com/review/VE1117941294.htm...,Variety,The film sports a provocative and appealing st...,2008-06-09 00:00:00,9559,Toy Story
4,Jonathan Rosenbaum,fresh,114709,http://onfilm.chicagoreader.com/movies/capsule...,Chicago Reader,"An entertaining computer-generated, hyperreali...",2008-03-10 00:00:00,9559,Toy Story


Clean the data. Retain only cases where fresh and quote are present and non-empty. Remove repeated observations (there are such in data). How many cases will be left?

In [18]:
rotten = rotten[rotten.fresh != "none"]
rotten = rotten.drop_duplicates()
print("Number of cases:", len(rotten))

Number of cases: 12823


Select training and validation data. You would like to split all cases into 80-20 groups.

We need to find the closest training quotes for each test quote, then predict if the movie is fresh or rotten according to the quote.

Following the same steps we did with the texts above:

(a) Create dictionary and BOW of all quotes

(b) Run k-NN with several different k's and predict if fresh or rotten. In each run, compute the accuracy.

(c) Transform the data into TF-IDF form and repeat k-NN.

(d) Inspect a few cases where the tomato was correctly and incorrectly predicted. Can we explain why the algorithm behaved in the way it behaved?


In [19]:
ntrain = 1500
nVal = 300

train_df = rotten.sample(n=ntrain, random_state = 123)
val_df = rotten.loc[~rotten.index.isin(train_df)].sample(n=nVal, random_state = 123)

X_train = train_df.quote
X_val = val_df.quote
y_train = train_df.fresh
y_val = val_df.fresh

sentences = np.concatenate((X_train.values, X_val.values),axis=0)

vectorizer = CountVectorizer(min_df=0)
vectorizer.fit(sentences)
X = vectorizer.transform(sentences).toarray()

Now that we've created dictionary and BOW of all quotes, let's run k-NN with several different k's and predict if fresh or rotten. In each run, compute the accuracy.
Essentially, we are doing the same thing we did above!

In [20]:
# BOW
train_cases = X[:ntrain]

for l in [1, 5, 25]:
    start = time.time()
    knn(l)
    end = time.time()
    print("Time elapsed:", end - start)

k: 1
Accuracy: 1.0
Time elapsed: 16.691444873809814
k: 5
Accuracy: 0.7533333333333333
Time elapsed: 17.427025318145752
k: 25
Accuracy: 0.6866666666666666
Time elapsed: 16.843358755111694


Transform the data into TF-IDF form and repeat k-NN.

In [21]:
# TD-IDF
tf = X.T # term frequency

idf = np.log(tf.shape[1] / ( 1 + sum(tf != 0) ))
idf = np.diag(idf)

tfidf = np.dot(tf,idf).T

print(tf.shape)
print(idf.shape)
print(tfidf.shape, '\n')

# normalize
tfidf = tfidf/np.sqrt(np.sum(tfidf**22))

# train datasets
train_tfidf = tfidf[:ntrain]

for l in [1,5,25]:
    start = time.time()
    knn_tfidf(l)
    end = time.time()
    print("Time elapsed:", end - start)

(6770, 1800)
(1800, 1800)
(1800, 6770) 

k: 1
Accuracy: 1.0
Time elapsed: 78.05132484436035
k: 5
Accuracy: 0.7533333333333333
Time elapsed: 78.02384424209595
k: 25
Accuracy: 0.6866666666666666
Time elapsed: 81.88916397094727


*What worked better: reviews or text pages? What worked better: BOW or TF-IDF?*

We can say once again that: both of our models are best when k = 1, with an accuracy of 1.0. Accuracy decreases as we increase k. BOW and TF-IDF seemed to have similar accuracies with the same k's again, just as we saw in the last section. When we compare reviews vs text pages, we see that with reviews, accuracy decreases along with k at a lesser rate than text pages, showing us that reviews may have worked better.